In [1]:
%load_ext autoreload
%autoreload 2
import main
import storage
import ranks
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import itertools
from itertools import takewhile,dropwhile
from fractions import Fraction
import pd_cols

c:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# df = storage.merge_and_load()
# df=df[~(df['note'] == 'ForcedSpecimen0')]
# storage.overwrite(df)
# 42

 C:\Users\Admin\AppData\Local\Temp\ipykernel_3592\618935152.py:3: FutureWarning:arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


42

In [18]:
# %matplotlib inline
%load_ext autoreload
%autoreload 2
import main, storage, ranks
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import itertools
from itertools import takewhile,dropwhile
from fractions import Fraction
import pd_cols

def p(func, /, *args, **keywords):
    def newfunc(*fargs, **fkeywords):
        newkeywords = {**keywords, **fkeywords}
        return func(*args, *fargs, **newkeywords)
    newfunc.func = func
    newfunc.args = args
    newfunc.keywords = keywords
    return newfunc

if False: #delete the all experiment records
        shutil.rmtree("ppdata")
        shutil.rmtree("exdata")
        shutil.rmtree("graphs")

df_og = storage.merge_and_load()
df_og = df_og[(df_og['scale_train'] == False)&(df_og['sort_train'] == False)]
df_og['full_desc'] = df_og.apply(pd_cols.get_full_desc,axis=1)
df_og, common_eval = ranks.compute_ranks(df_og)

# helper functions and axilliary data 
pd.options.mode.chained_assignment = None          # prevents displaying a useless warning

def df_enhance(df):
        df['avg_rank'] = df['ranks'].apply(np.mean) 
        df['last_rank'] = df['ranks'].apply(lambda a: a[-1]) 
        df['true_evaluations'] = (df['pop_size'] * df['true_ratio']).map(int)
        df['true_ratio'] = df['true_ratio'].map(Fraction).map(lambda f:Fraction(1,12) if f.denominator > 9999 else f) # due to floating point inacurracies the 1/12 gets represented weird and needs to be repaired 
        df['dim_red'] = df['dim_red'].replace('','none')
        df['model'] = df['model'].replace('','none')
        df['dim_red_kind'] = df['dim_red'].map(lambda a: ''.join(takewhile(lambda s: s.isalpha(), a)))
        df['model_kind'] = df['model'].map(lambda a: ''.join(takewhile(lambda s: s.isalpha(), a)))
        return df

df_og = df_enhance(df_og)
# df_og = df_og[df_og['scale_train']]
pure_mask = df_og['evo_mode'].map(str) == 'Pure'
pures = df_og[pure_mask] 
# pures2 = df_og[df_og['scale_train']==Tru] 

pca_mask = (df_og['model_kind'] == 'gp')&(df_og['dim_red_kind'] == 'pca')&((df_og['pop_size']==48)|(df_og['pop_size']==64))&(df_og['true_ratio'].map(Fraction)==Fraction(1/8))
pca_df = df_og[pca_mask] 

# keep the special cases separate, it makes all the graphing easier
df_og = df_og[~(
       (pure_mask &(df_og['pop_size'] != 8))
    #    |(pca_mask&(df_og['dim_red'] != 'pca0.5'))   
)]  





The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


 C:\Users\Admin\AppData\Local\Temp\ipykernel_9640\2579267596.py:28: FutureWarning:arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


In [19]:
def two_layer_tics(ax):
        plt.xticks(rotation=0, size= 'xx-small')
        for tick in ax.xaxis.get_major_ticks()[1::2]:
                tick.set_pad(15)
def save_and_show(name:str):
        plt.savefig(f'graphs/{name}.png', bbox_inches='tight')
        plt.show()
def print_latex(df: pd.Series):
        r = lambda s: ' ' if r is None else s.replace('_', ' ')
        s= "\\begin{tabular}{|lr|}\n"  # + " | ".join(["c"] * len(df.columns)) + "}\n"
        s+= "\\hline\n"
        s+= r(df.index.name) + ' & ' + r(df.name) +'\\\\\n'
        s+= "\\hline\n"
        for k, v in df.items():
                s+= f"{k} & {v:0.2f} \\\\\n"
        s+= "\\hline\n"
        s+= "\\end{tabular}"
        print(s)
def np_apply_axis0(fn=None):
        def inner(arr, fn):
                arr = arr.to_list()
                b = np.apply_along_axis(fn,0,arr) 
                return list(b)
        return lambda a: inner(a, fn)
def close_to(series, num):
        return series.map(lambda a: abs(a - num) <= 1e-3)

def default_groupby(df, columns):
        map_dict = {
                'ranks':np_apply_axis0(np.average), 
                'avg_rank':'mean', 
                'last_rank':'mean',
                'elapsed_time':'mean',
                'model':'first',
                'dim_red':'first',
                'model_kind':'first',
                'dim_red_kind':'first',
        }
        for c in columns:  
               if c in map_dict: 
                      del map_dict[c]
        res = df.groupby(columns).agg(map_dict)
        return res

baselines = default_groupby(pures, ['pop_size'])
baseline_color = '#E04836'
default_color = 'forestgreen'
def bar(df,x_name=None, y_name=None,  index_mapper = None, y_mapper = None, regr = False, baseline_i=-1, x_ticklabel_mapper=None, print_table=True):
        if x_name != None:    
            df = default_groupby(df,x_name)
        if y_name == None:    
            y_name='last_rank'
        df = df.sort_index()
        
        if print_table:
                print_latex(df[y_name])
        colors = [default_color for _ in range(len(df))]
        if baseline_i != -1 :
                colors += [baseline_color]
                df.loc[str(len(df))] = baselines.loc[baseline_i]
                
        x = df.index
        y = df[y_name] 
        
        # if index_mapper != None: 
        #        x = x.map(index_mapper)
        # if y_mapper != None: 
        #        y = y.map(y_mapper)
        ax = sns.barplot(x=x.map(str), y=y, palette=colors)
        # fig, ax = plt.subplots()
        # ax.bar(x.map(str), y, label=bar_labels, color=bar_colors)
        ax.set_ylabel('Rank Percentile')
        xn = df.index.name if x_name == None else x_name
        if xn != None: # df.index.name can be None
                xn = xn.split('_')
                xn = map(lambda s: s.capitalize(),xn)
                xn = ' '.join(xn)
                ax.set_xlabel(xn)
        xticklabels = ax.get_xticklabels()
        if x_ticklabel_mapper:
               xticklabels = x_ticklabel_mapper(xticklabels)
        if baseline_i != -1 :
                xticklabels[-1] = 'baseline'
                plt.axhline(y=y[-1], color=baseline_color, linestyle='dotted')
        ax.set_xticklabels(xticklabels, size='small')
        # from pydoc import locate
# >>> locate('int')
        # print('\n'.join(map(lambda (a,b): f'{a}',zip(y,x))))
        if regr:
                xx = np.arange(len(y)-(1 if baseline_i != -1  else 0))
                m, b = np.polyfit(xx, y[:-1] if baseline_i != -1  else y, 1)
                ax.plot(xx, m*xx + b,color='red', alpha=0.5)
        return ax 



In [20]:
df = df_og
ax = bar(df,'full_desc',)
for tick in ax.xaxis.get_major_ticks()[1::2]:
    tick.set_pad(15)
plt.xticks(size= 5)
save_and_show('rrrrr')

\begin{tabular}{|lr|}
\hline
full desc & last rank\\
\hline
8_Doe_Pure&10_8_100_300_201_no_mean & 33.29 \\
8_Doe_Pure&10_8_200_300_200_sig_sampleinc2 & 36.64 \\
8_Doe_Pure&10_8_40_300_200 & 47.12 \\
8_Doe_Pure&10_8_40_300_200_sig & 42.01 \\
8_Doe_Pure&10_8_40_300_200_sig_results & 50.67 \\
8_Doe_Pure&10_8_40_300_200_sig_sampleinc & 37.01 \\
8_Doe_Pure&10_8_40_300_200_sig_sampleinc2 & 31.03 \\
8_Pure_300 & 63.89 \\
8_Pure_300_sig & 64.17 \\
\hline
\end{tabular}


 C:\Users\Admin\AppData\Local\Temp\ipykernel_9640\4100544346.py:69: FutureWarning:

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

 C:\Users\Admin\AppData\Local\Temp\ipykernel_9640\4100544346.py:85: UserWarning:set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.


In [15]:
df = df_og
ax = bar(df,'full_desc',)
for tick in ax.xaxis.get_major_ticks()[1::2]:
    tick.set_pad(15)
plt.xticks(size= 5)
save_and_show('rrrrr')

\begin{tabular}{|lr|}
\hline
full desc & last rank\\
\hline
10_Pure_300_-1_False_False_False & 80.35 \\
11_Pure_300_-1_False_False_False & 73.75 \\
12_Pure_300_-1_False_False_False & 70.87 \\
13_Pure_300_-1_False_False_False & 70.21 \\
14_Pure_300_-1_False_False_False & 68.54 \\
15_Pure_300_-1_False_False_False & 65.69 \\
16_Pure_300_-1_False_False_False & 63.02 \\
17_Pure_300_-1_False_False_False & 60.45 \\
18_Pure_300_-1_False_False_False & 55.62 \\
19_Pure_300_-1_False_False_False & 56.84 \\
20_Pure_300_-1_False_False_False & 54.31 \\
21_Pure_300_-1_False_False_False & 58.40 \\
22_Pure_300_-1_False_False_False & 46.70 \\
23_Pure_300_-1_False_False_False & 46.88 \\
24_Pure_300_-1_False_False_False & 47.19 \\
25_Pure_300_-1_False_False_False & 44.69 \\
26_Pure_300_-1_False_False_False & 43.47 \\
27_Pure_300_-1_False_False_False & 46.53 \\
28_Pure_300_-1_False_False_False & 40.94 \\
29_Pure_300_-1_False_False_False & 37.43 \\
2_Pure_300_-1_False_False_False & 46.01 \\
30_Pure_300_-1_Fa

 C:\Users\Admin\AppData\Local\Temp\ipykernel_4128\4100544346.py:69: FutureWarning:

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

 C:\Users\Admin\AppData\Local\Temp\ipykernel_4128\4100544346.py:85: UserWarning:set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.


In [6]:
df = df_og
df = default_groupby(df, ['true_ratio'])
ax = bar(df,'true_ratio')

save_and_show('rrrrr')

\begin{tabular}{|lr|}
\hline
true ratio & last rank\\
\hline
1 & 50.00 \\
\hline
\end{tabular}


 C:\Users\Admin\AppData\Local\Temp\ipykernel_4128\4100544346.py:69: FutureWarning:

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

 C:\Users\Admin\AppData\Local\Temp\ipykernel_4128\4100544346.py:85: UserWarning:set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.


In [11]:
df = pures
# df = df[df['pop_size']]
ax = bar(df, 'pop_size')
ax.set_xlabel('Population Size')
ax.set_title('Normal Evaluation')

save_and_show('pure')

\begin{tabular}{|lr|}
\hline
pop size & last rank\\
\hline
2 & 31.69 \\
4 & 60.35 \\
6 & 71.11 \\
8 & 68.42 \\
16 & 48.88 \\
32 & 25.57 \\
\hline
\end{tabular}


 C:\Users\Admin\AppData\Local\Temp\ipykernel_4952\4100544346.py:69: FutureWarning:

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

 C:\Users\Admin\AppData\Local\Temp\ipykernel_4952\4100544346.py:85: UserWarning:set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.


In [12]:
# pca 
from itertools import takewhile,dropwhile

st =False
df=pca_df[pca_df['scale_train']==st]
df = df[(df['model_kind'] == 'gp')&(df['pop_size']==64)]

df['pca_ratio'] = df['dim_red'].map(lambda a: ''.join(dropwhile(lambda s: s.isalpha(), a)))

# df1 = df_og[(df_og['model_kind'] == 'gp')&(df_og['dim_red_kind'] == 'none')&(df_og['pop_size']==48)&(df_og['true_ratio'].map(Fraction)==Fraction(1/8))].iloc[0]
# df1['pca_ratio'] = str(1.0)
# df.loc[str(len(df))] = df1
df1 = df_og[(df_og['model_kind'] == 'gp')&(df_og['dim_red_kind'] == 'none')&(df_og['pop_size']==64)&(df_og['true_ratio'].map(Fraction)==Fraction(1/8))].iloc[0]

df1['pca_ratio'] = str(1.0)
df.loc[str(len(df))] = df1
ax = bar(df,'pca_ratio', regr=True, baseline_i=8)
ax.set_xlabel('pca reduction ratio')
ax.set_title('PCA + GP')

save_and_show('pca')

\begin{tabular}{|lr|}
\hline
pca ratio & last rank\\
\hline
0.1 & 55.75 \\
0.2 & 47.76 \\
0.3 & 55.65 \\
0.4 & 57.11 \\
0.5 & 63.26 \\
0.6 & 66.13 \\
0.7 & 65.43 \\
0.8 & 73.28 \\
0.9 & 72.91 \\
1.0 & 93.40 \\
\hline
\end{tabular}


 C:\Users\Admin\AppData\Local\Temp\ipykernel_4952\4100544346.py:69: FutureWarning:

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

 C:\Users\Admin\AppData\Local\Temp\ipykernel_4952\4100544346.py:84: FutureWarning:Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
 C:\Users\Admin\AppData\Local\Temp\ipykernel_4952\4100544346.py:85: UserWarning:set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.


In [14]:
df=df_og
df = df[(df['model_kind'] == 'gp')& (df['dim_red_kind'] == 'none')]  
df = default_groupby(df, ['true_evaluations', 'pop_size'])
pures2 = pures.set_index(pures['pop_size'].map(lambda n: (n,n)))
df = pd.concat([df, pures2])
ax = bar(df, print_table=False)

save_and_show('pop_evals')

 C:\Users\Admin\AppData\Local\Temp\ipykernel_4952\4100544346.py:69: FutureWarning:

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

 C:\Users\Admin\AppData\Local\Temp\ipykernel_4952\4100544346.py:69: UserWarning:The palette list has more values (699928) than needed (22), which may not be intended.
 C:\Users\Admin\AppData\Local\Temp\ipykernel_4952\4100544346.py:85: UserWarning:set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.


: 

In [23]:
df=df_og
df = df[(df['model_kind'] == 'gp')]  
ax = bar(df, 'dim_red')
save_and_show('gp_dim_red')


\begin{tabular}{|lr|}
\hline
dim red & last rank\\
\hline
none & 69.78 \\
pca0.5 & 55.46 \\
vae[0.5] & 47.02 \\
\hline
\end{tabular}


 C:\Users\Admin\AppData\Local\Temp\ipykernel_7092\4100544346.py:69: FutureWarning:

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

 C:\Users\Admin\AppData\Local\Temp\ipykernel_7092\4100544346.py:85: UserWarning:set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.


In [22]:
df=df_og
ax = bar(df, 'dim_red_kind', 'elapsed_time')
ax.set_ylabel('Iteration Time (ms)')
save_and_show('elapsed')

\begin{tabular}{|lr|}
\hline
dim red kind & elapsed time\\
\hline
none & 5.70 \\
pca & 7.74 \\
vae & 50.63 \\
\hline
\end{tabular}


 C:\Users\Admin\AppData\Local\Temp\ipykernel_7092\4100544346.py:69: FutureWarning:

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

 C:\Users\Admin\AppData\Local\Temp\ipykernel_7092\4100544346.py:85: UserWarning:set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.


In [18]:
df=df_og
df = df[(df['dim_red_kind'] == 'none')&(df['pop_size']==48)&(df['true_ratio'].map(Fraction)==Fraction(1/8))]  
ax = bar(df, 'model')
two_layer_tics(ax)
save_and_show('models')



\begin{tabular}{|lr|}
\hline
model & last rank\\
\hline
elm10 & 59.43 \\
elm100 & 66.96 \\
elm150 & 63.57 \\
elm200 & 62.38 \\
elm30 & 68.40 \\
elm50 & 69.22 \\
elm500 & 52.35 \\
gp & 76.51 \\
mlp[0.5] & 12.13 \\
mlp[1, 1, 1] & 14.05 \\
mlp[1, 1] & 20.10 \\
mlp[10, 10, 10] & 28.28 \\
mlp[10, 10] & 26.08 \\
mlp[100] & 25.66 \\
mlp[10] & 17.00 \\
mlp[1] & 9.81 \\
mlp[2] & 11.96 \\
rbf[0.5]_1.0 & 17.97 \\
rbf[0.5]_2.0 & 27.25 \\
rbf[1]_1.0 & 18.03 \\
rbf[1]_10.0 & 68.48 \\
rbf[1]_5.0 & 60.54 \\
rbf[2]_0.5 & 15.41 \\
rbf[2]_1.0 & 21.60 \\
rbf[5]_5.0 & 64.16 \\
\hline
\end{tabular}


 C:\Users\Admin\AppData\Local\Temp\ipykernel_7092\4100544346.py:69: FutureWarning:

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

 C:\Users\Admin\AppData\Local\Temp\ipykernel_7092\4100544346.py:85: UserWarning:set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.


In [17]:
# dim_red
df=df_og
ax = bar(df, 'dim_red_kind')
save_and_show('dim_red_kind')


\begin{tabular}{|lr|}
\hline
dim red kind & last rank\\
\hline
none & 68.76 \\
pca & 55.46 \\
vae & 47.02 \\
\hline
\end{tabular}


 C:\Users\Admin\AppData\Local\Temp\ipykernel_7092\4100544346.py:69: FutureWarning:

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

 C:\Users\Admin\AppData\Local\Temp\ipykernel_7092\4100544346.py:85: UserWarning:set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.


In [10]:
# rank models without nodimred 
df=df_og
df1 = df[ (df['true_evaluations']==6) & (df['dim_red_kind'] == 'none')]  
ax = bar(df, 'model')
plt.xticks(rotation=90, size= 'small')
save_and_show('models')


\begin{tabular}{|lr|}
\hline
model & last rank\\
\hline
elm10 & 59.43 \\
elm100 & 66.96 \\
elm150 & 63.57 \\
elm200 & 62.38 \\
elm30 & 68.40 \\
elm50 & 69.22 \\
elm500 & 52.35 \\
gp & 56.56 \\
mlp[0.5] & 12.13 \\
mlp[1, 1, 1] & 14.05 \\
mlp[1, 1] & 20.10 \\
mlp[10, 10, 10] & 28.28 \\
mlp[10, 10] & 26.08 \\
mlp[100] & 25.66 \\
mlp[10] & 17.00 \\
mlp[1] & 9.81 \\
mlp[2] & 11.96 \\
rbf[0.5]_1.0 & 18.00 \\
rbf[0.5]_2.0 & 27.25 \\
rbf[1]_1.0 & 18.03 \\
rbf[1]_10.0 & 68.48 \\
rbf[1]_5.0 & 60.54 \\
rbf[2]_0.5 & 15.41 \\
rbf[2]_1.0 & 21.60 \\
rbf[5]_5.0 & 64.16 \\
\hline
\end{tabular}


 C:\Users\Admin\AppData\Local\Temp\ipykernel_7092\4100544346.py:69: FutureWarning:

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

 C:\Users\Admin\AppData\Local\Temp\ipykernel_7092\4100544346.py:85: UserWarning:set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.


In [13]:
#popsize,true_evals           with only gp
from fractions import Fraction
df=df_og
df=df[(df['model_kind'] == 'gp')&(df['dim_red_kind']=='none')]

ax = bar(df, 'true_ratio',index_mapper = lambda a: Fraction(a))
ax.set_label('dim red, true evals, aux evals')
ax.set_xlabel('truly evaluated fraction of population')
ax.set_ylabel('rank percentile avg')

labels = [item.get_text() for item in ax.get_xticklabels()]
labels[1] = '1/12'
ax.set_xticklabels(labels)
save_and_show('popsize_evals')

\begin{tabular}{|lr|}
\hline
true ratio & last rank\\
\hline
1/16 & 73.90 \\
1/12 & 76.95 \\
1/8 & 75.01 \\
1/4 & 72.66 \\
1/2 & 66.72 \\
\hline
\end{tabular}


 C:\Users\Admin\AppData\Local\Temp\ipykernel_7092\4100544346.py:69: FutureWarning:

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

 C:\Users\Admin\AppData\Local\Temp\ipykernel_7092\4100544346.py:85: UserWarning:set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
 C:\Users\Admin\AppData\Local\Temp\ipykernel_7092\1393360363.py:13: UserWarning:set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.


In [15]:
#popsize,true_evals           with only gp
df=df_og
df=df[(df['model_kind'] == 'gp')&(df['dim_red_kind']=='none')]
df = default_groupby(df, ['true_evaluations','pop_size'])
ax = bar(df)
ax.set_label('dim red, true evals, aux evals')
ax.set_xlabel('population: (true evaluated, generated)')
ax.set_ylabel('rank percentile avg')
# xlabel = ax.get_xlabel()
# ax.set_xlabel([1,2,3], rotation='horizontal')
# plt.savefig("graphs/pop.png")
# plt.pause(0.01)
plt.show() 

AttributeError: 'NoneType' object has no attribute 'replace'

In [ ]:
# df=pca_df[(pca_df['scale_train']==True)&(pca_df['function']==8)&(pca_df['instance']==1)]
df = df_og
ranks.plot_ranks(df, common_eval)

 c:\Users\Admin\Desktop\Diplomka\plotting.py:103: FutureWarning:Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
